# Movie Reccomendation System

In [268]:
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import LabelBinarizer


In [269]:
movies = pd.read_csv('Data/movies.csv')
links = pd.read_csv('Data/links.csv')
ratings = pd.read_csv('Data/ratings.csv')
tags = pd.read_csv('Data/tags.csv')

In [270]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


### Movie Cleaning

In [278]:
movies[450:575]

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
450,515,"Remains of the Day, The (1993)","[Drama, Romance]",0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
451,516,Renaissance Man (1994),"[Comedy, Drama]",0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
452,517,Rising Sun (1993),"[Action, Drama, Mystery]",0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
453,518,"Road to Wellville, The (1994)",[Comedy],0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,519,RoboCop 3 (1993),"[Action, Crime, Drama, Sci-Fi, Thriller]",0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,697,Feeling Minnesota (1996),"[Drama, Romance]",0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
571,698,Delta of Venus (1995),[Drama],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
572,700,Angus (1995),[Comedy],0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
573,703,Boys (1996),[Drama],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## One-hot Encoding the Genres

In [272]:
def unpack_genres(string):
    s = string.split('|')
    return s

test = pd.DataFrame()
test['genres'] = movies['genres'].map(unpack_genres)
movies['genres'] = movies['genres'].map(unpack_genres)

In [273]:
unique_genres = [ ]
for row in test['genres']:
    for genre in row:
        if genre not in unique_genres:
            unique_genres.append(genre)

unique_genres

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'War',
 'Musical',
 'Documentary',
 'IMAX',
 'Western',
 'Film-Noir',
 '(no genres listed)']

In [274]:
for genre in unique_genres:
    movies[genre] = movies['genres'].apply(lambda x: 1 if genre in x else 0)

# Drop the original 'Categories' column
df = movies.drop('genres', axis=1)

df.head()

,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
